In [1]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow import keras
import numpy as np
from os import listdir
from os.path import isfile
import math

from datetime import datetime
%load_ext tensorboard

In [2]:
class SequenceGenerator(keras.utils.Sequence):
    """
    A keras Sequence to be used as an image generator for the model.
    """

    def __init__(self, x, y, batchsize):
        self.x, self.y, self.batchsize = x, y, batchsize

    def __len__(self):
        return math.ceil(len(self.x) / self.batchsize)

    def __getitem__(self, idx):
        x_names = self.x[idx * self.batchsize:(idx + 1) * self.batchsize]
        y_names = np.asarray(self.y[idx * self.batchsize:(idx + 1) * self.batchsize])
        
        # open x image names, resize, normalise and make a numpy array
        batch_x = np.asarray([preprocess_input(img_to_array(load_img(file_name, target_size=(299, 299)))) for file_name in x_names])

        return batch_x, y_names

    def num_classes(self):
        ret = []
        for cat in self.y:
            if cat not in ret:
                ret.append(cat)
        return len(ret)

In [3]:
def get_filenames_from_dir(directory, num_categories=1000):
    x = []
    y = []
    category_count = 0
    for category in listdir(directory):
        if isfile(category):
            continue
        for file in listdir("{0}/{1}".format(directory, category)):
            if file[-3:] != "jpg":
                continue
            x.append("{0}/{1}/{2}".format(directory, category, file))
            y.append(category_count)
        category_count += 1
        if category_count == num_categories:
            break

    return x, y

In [4]:
# makes arrays of the images and label names
x_names, y_names = get_filenames_from_dir("database")

# 15% of all the images are set aside as the test set
x_train_val, x_test, y_train_val, y_test = train_test_split(x_names, y_names, test_size=0.15, random_state=42)

# 17% of the non-test images are set aside as the validation set
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.17, random_state=42)

In [5]:
# make generators with batch size 4 for each set
train_gen = SequenceGenerator(x_train, y_train, 4)
val_gen = SequenceGenerator(x_val, y_val, 4)
test_gen = SequenceGenerator(x_test, y_test, 4)

In [6]:
# Inception model
base_model = InceptionV3(weights="imagenet")
for layer in base_model.layers:
    layer.trainable = True

predictions = keras.layers.Dense(train_gen.num_classes(), activation='softmax')(base_model.layers[-2].output)

model = keras.Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
logdir = "logs/inception_single_model" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [8]:
model.fit(train_gen, validation_data=val_gen, callbacks=[tensorboard_callback], epochs=20)

Epoch 1/20
11176/11176 [==============================] - 855s 75ms/step - loss: 5.1841 - accuracy: 0.0241 - val_loss: 4.7419 - val_accuracy: 0.0482
Epoch 2/20
11176/11176 [==============================] - 848s 76ms/step - loss: 4.0778 - accuracy: 0.0936 - val_loss: 3.3002 - val_accuracy: 0.1720
Epoch 3/20
11176/11176 [==============================] - 834s 75ms/step - loss: 2.5287 - accuracy: 0.3362 - val_loss: 1.9761 - val_accuracy: 0.4290
Epoch 4/20
11176/11176 [==============================] - 842s 75ms/step - loss: 1.5154 - accuracy: 0.5579 - val_loss: 1.4569 - val_accuracy: 0.5751
Epoch 5/20
11176/11176 [==============================] - 830s 74ms/step - loss: 1.0611 - accuracy: 0.6808 - val_loss: 1.3282 - val_accuracy: 0.6228
Epoch 6/20
11176/11176 [==============================] - 849s 76ms/step - loss: 0.7950 - accuracy: 0.7542 - val_loss: 0.8862 - val_accuracy: 0.7335
Epoch 7/20
11176/11176 [==============================] - 829s 74ms/step - loss: 0.6111 - accuracy: 0.8091

In [9]:
model.evaluate(test_gen)

2377/2377 [==============================] - 61s 26ms/step - loss: 0.9384 - accuracy: 0.8113


[0.9384143948554993, 0.8112572431564331]